## Imports and Data Loading

In [1]:
import gc
import os
import random
import warnings

import numpy as np
import pandas as pd
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import f1_score
from sklearn.model_selection import GroupKFold
from tensorflow.keras import backend as K
from tensorflow.keras import losses, models, optimizers
from tensorflow.keras.activations import sigmoid
from tensorflow.keras.callbacks import Callback, LearningRateScheduler
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam

import warnings
warnings.filterwarnings('ignore')

GROUP_BATCH_SIZE = 4000
EPOCHS = 30  # toy data
BATCHSIZE = 64
LR = 0.0015
SPLITS = 6
SEED = 42


def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)


def read_data(kalman):
    if kalman:  # if use kalman filtered data
        train = pd.read_csv(
            '/kaggle/input/ion-switching-kalman-data/train_clean_kalman.csv',
            dtype={
                'time': np.float32,
                'signal': np.float32,
                'open_channels': np.int32
            })
        test = pd.read_csv(
            '/kaggle/input/ion-switching-kalman-data/test_clean_kalman.csv',
            dtype={
                'time': np.float32,
                'signal': np.float32
            })
    else:
        train = pd.read_csv('/kaggle/input/data-without-drift/train_clean.csv',
                            dtype={
                                'time': np.float32,
                                'signal': np.float32,
                                'open_channels': np.int32
                            })
        test = pd.read_csv('/kaggle/input/data-without-drift/test_clean.csv',
                           dtype={
                               'time': np.float32,
                               'signal': np.float32
                           })

    sub = pd.read_csv(
        '/kaggle/input/liverpool-ion-switching/sample_submission.csv',
        dtype={'time': np.float32})

    Y_train_proba = np.load(
        "/kaggle/input/ion-shifted-rfc-proba/Y_train_proba.npy")
    Y_test_proba = np.load(
        "/kaggle/input/ion-shifted-rfc-proba/Y_test_proba.npy")
    for i in range(11):
        train[f"proba_{i}"] = Y_train_proba[:, i]
        test[f"proba_{i}"] = Y_test_proba[:, i]
    return train, test, sub


Using TensorFlow backend.


## Feature Engineering

In [2]:
def batching(df, batch_size):  # take every 4000 signals as a sample 
    df['group'] = df.groupby(df.index // batch_size,
                             sort=False)['signal'].agg(['ngroup']).values
    df['group'] = df['group'].astype(np.uint16)
    return df


def normalize(train, test):
    train_input_mean = train.signal.mean()
    train_input_sigma = train.signal.std()
    train['signal'] = (train.signal - train_input_mean) / train_input_sigma
    test['signal'] = (test.signal - train_input_mean) / train_input_sigma
    return train, test


def lag_with_pct_change(df, windows):    # simple FE
    for w in windows:
        df['signal_shift_pos_' +
           str(w)] = df.groupby('group')['signal'].shift(w).fillna(0)
        df['signal_shift_neg_' + str(w)] = df.groupby('group')['signal'].shift(
            -1 * w).fillna(0)
    return df


def run_feat_engineering(df, batch_size):
    df = batching(df, batch_size=batch_size)
    df = lag_with_pct_change(df, [1, 2, 3])
    df['signal_2'] = df['signal']**2
    return df


def feature_selection(train, test):
    features = [
        col for col in train.columns
        if col not in ['index', 'group', 'open_channels', 'time']
    ]
    train = train.replace([np.inf, -np.inf], np.nan)
    test = test.replace([np.inf, -np.inf], np.nan)

    for feature in features:
        feature_mean = pd.concat([train[feature], test[feature]],
                                 axis=0).mean()
        train[feature] = train[feature].fillna(feature_mean)
        test[feature] = test[feature].fillna(feature_mean)
    print('Using the following features')
    print(features)
    return train, test, features


## WaveNet Block

In [3]:
def wave_block(x, filters, kernel_size, n): # definition of wavenet
    dilation_rates = [2**i for i in range(n)]
    x = Conv1D(filters=filters, kernel_size=1, padding='same')(x)
    res_x = x
    for dilation_rate in dilation_rates:
        tanh_out = Conv1D(filters=filters,
                          kernel_size=kernel_size,
                          padding='same',
                          activation='tanh',
                          dilation_rate=dilation_rate)(x)
        sigm_out = Conv1D(filters=filters,
                          kernel_size=kernel_size,
                          padding='same',
                          activation='sigmoid',
                          dilation_rate=dilation_rate)(x)
        x = Multiply()([tanh_out, sigm_out])
        x = Conv1D(filters=filters, kernel_size=1, padding='same')(x)
        res_x = Add()([res_x, x])
    return res_x

## Model

In [4]:
def Wavenet_LSTM_Classifier(shape_):
    inp = Input(shape=(shape_))
    x = Conv1D(64,
               kernel_size=7,
               dilation_rate=1,
               strides=1,
               padding="same")(inp)
    x = BatchNormalization()(x)
    x *= sigmoid(x)

    x1 = wave_block(x, 16, 3, 12)
    x2 = Bidirectional(LSTM(16, return_sequences=True))(x)
    x = tf.keras.layers.concatenate([x1, x2], axis=2)
    x = BatchNormalization()(x)

    x1 = wave_block(x, 32, 3, 8)
    x2 = Bidirectional(LSTM(32, return_sequences=True))(x)
    x = tf.keras.layers.concatenate([x1, x2], axis=2)
    x = BatchNormalization()(x)

    x1 = wave_block(x, 64, 3, 4)
    x2 = Bidirectional(LSTM(64, return_sequences=True))(x)
    x = tf.keras.layers.concatenate([x1, x2], axis=2)
    x = BatchNormalization()(x)

    x1 = wave_block(x, 128, 3, 1)
    x2 = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = tf.keras.layers.concatenate([x1, x2], axis=2)
    x = BatchNormalization()(x)

    x = Dropout(0.2)(x)
    out = Dense(11, activation='softmax', name='out')(x)

    model = models.Model(inputs=inp, outputs=out)

    opt = Adam(lr=LR)
    model.compile(loss=losses.CategoricalCrossentropy(),
                  optimizer=opt,
                  metrics=['accuracy'])
    tf.keras.utils.plot_model(model,
                              to_file="Wavenet_LSTM_Classifier.png",
                              show_shapes=False,
                              show_layer_names=True,
                              rankdir="LR",
                              expand_nested=False,
                              dpi=96
                            )
    return model


## Traning

In [5]:
class MacroF1(Callback):
    def __init__(self, model, inputs, targets):
        self.model = model
        self.inputs = inputs
        self.targets = np.argmax(targets, axis=2).reshape(-1)

    def on_epoch_end(self, epoch, logs):
        pred = np.argmax(self.model.predict(self.inputs), axis=2).reshape(-1)
        score = f1_score(self.targets, pred, average='macro')
        print('F1 Macro Score: %s' % score)

def lr_schedule(epoch):
    if epoch < 30:
        lr = LR
    elif epoch < 40:
        lr = LR / 3
    elif epoch < 50:
        lr = LR / 5
    elif epoch < 60:
        lr = LR / 7
    elif epoch < 70:
        lr = LR / 9
    elif epoch < 80:
        lr = LR / 11
    elif epoch < 90:
        lr = LR / 13
    else:
        lr = LR / 100
    return lr

def run_cv_nn(train, test, splits, feats, nn_epochs, nn_batch_size, filename):
    seed_everything(SEED)
    K.clear_session()
    config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,
                                      inter_op_parallelism_threads=1)
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(),
                                config=config)
    tf.compat.v1.keras.backend.set_session(sess)

    oof_ = np.zeros((len(train), 11))
    preds_ = np.zeros((len(test), 11))
    target = ['open_channels']
    group = train['group']

    kf = GroupKFold(n_splits=splits)
    splits = [x for x in kf.split(train, train[target], group)]
    splits = list(
        map(lambda x: [np.unique(group[x[0]]),
                       np.unique(group[x[1]]), x[1]], splits))
    tr = pd.concat([pd.get_dummies(train.open_channels), train[['group']]],
                   axis=1)

    tr.columns = ['target_' + str(i) for i in range(11)] + ['group']
    target_cols = ['target_' + str(i) for i in range(11)]
    train_tr = np.array(
        list(tr.groupby('group').apply(
            lambda x: x[target_cols].values))).astype(np.float32)

    train = np.array(
        list(train.groupby('group').apply(lambda x: x[feats].values)))
    test = np.array(
        list(test.groupby('group').apply(lambda x: x[feats].values)))

    for n_fold, (tr_idx, val_idx, val_orig_idx) in enumerate(splits[0:],
                                                             start=0):
        train_x, train_y = train[tr_idx], train_tr[tr_idx]
        valid_x, valid_y = train[val_idx], train_tr[val_idx]

        gc.collect()
        shape_ = (None, train_x.shape[2])
        model = Wavenet_LSTM_Classifier(shape_)

        checkpoint = ModelCheckpoint(filepath='/kaggle/output/%s.h5' %
                                     filename,
                                     monitor=MacroF1(model, valid_x, valid_y),
                                     mode='max',
                                     save_best_only='True')
        cb_lr_schedule = LearningRateScheduler(lr_schedule)
        model.fit(train_x,
                  train_y,
                  epochs=nn_epochs,
                  callbacks=[
                      cb_lr_schedule,
                      MacroF1(model, valid_x, valid_y), checkpoint
                  ],
                  batch_size=nn_batch_size,
                  verbose=1,
                  validation_data=(valid_x, valid_y))

        preds_f = model.predict(valid_x)
        f1_score_ = f1_score(np.argmax(valid_y, axis=2).reshape(-1),
                             np.argmax(preds_f, axis=2).reshape(-1),
                             average='macro')
        model.save("/kaggle/working/%s_%s.h5" % (filename, f1_score_))
        print('Training fold %s completed. macro f1 score : %s' %
              (n_fold + 1,f1_score_))

        preds_f = preds_f.reshape(-1, preds_f.shape[-1])
        oof_[val_orig_idx, :] += preds_f

        te_preds = model.predict(test)
        te_preds = te_preds.reshape(-1, te_preds.shape[-1])
        preds_ += te_preds / SPLITS

    f1_score_ = f1_score(np.argmax(train_tr, axis=2).reshape(-1),
                         np.argmax(oof_, axis=1).reshape(-1),
                         average='macro')
    print('Training completed. oof macro f1 score : %s' % f1_score_)
    np.save('/kaggle/working/%s_predicted_test.npy' % (round(f1_score_, 6)), preds_)


## Applying all steps to kalman and non_kalman data

In [6]:
for kalman, model_name in zip([True, False], ['kalman', 'non_kalman']):
    train, test, sample_submission = read_data(kalman)
    train, test = normalize(train, test)
    train = run_feat_engineering(train, batch_size=GROUP_BATCH_SIZE)
    test = run_feat_engineering(test, batch_size=GROUP_BATCH_SIZE)
    train, test, features = feature_selection(train, test)

    gc.collect()
    run_cv_nn(train, test, SPLITS, features, EPOCHS, BATCHSIZE, model_name)

Using the following features
['signal', 'proba_0', 'proba_1', 'proba_2', 'proba_3', 'proba_4', 'proba_5', 'proba_6', 'proba_7', 'proba_8', 'proba_9', 'proba_10', 'signal_shift_pos_1', 'signal_shift_neg_1', 'signal_shift_pos_2', 'signal_shift_neg_2', 'signal_shift_pos_3', 'signal_shift_neg_3', 'signal_2']
Train on 1041 samples, validate on 209 samples
Epoch 1/30
1041/1041 [==============================] - 70s 67ms/sample - loss: 0.8782 - accuracy: 0.7116 - val_loss: 1.3042 - val_accuracy: 0.6180
Epoch 2/30
1041/1041 [==============================] - 37s 35ms/sample - loss: 0.3636 - accuracy: 0.8649 - val_loss: 1.1846 - val_accuracy: 0.7465
Epoch 3/30
1041/1041 [==============================] - 37s 35ms/sample - loss: 0.2178 - accuracy: 0.9251 - val_loss: 1.0406 - val_accuracy: 0.8487
Epoch 4/30
1041/1041 [==============================] - 37s 35ms/sample - loss: 0.1569 - accuracy: 0.9498 - val_loss: 0.9000 - val_accuracy: 0.8763
Epoch 5/30
1041/1041 [==============================] -

## Blending

In [7]:
pre_npy = os.listdir('/kaggle/working/')
pre_npy=[x for x in pre_npy if '.npy' in x]
result = np.zeros((2000000, 11))
# weights=[np.random.random() for _ in range(len(pre_npy))]
weights = np.ones(len(pre_npy))

weights /= np.sum(weights)
for weight, pre_f in zip(weights, pre_npy):
    result += np.load("/kaggle/working/%s" % pre_f,) * weight

sample_submission['open_channels'] = np.argmax(result, axis = 1).astype(int)
sample_submission.to_csv('submission.csv', index=False)